In [1]:
from datetime import datetime
from datetime import date
from bs4 import BeautifulSoup

from dateutil.relativedelta import relativedelta 
import requests
import time
import re

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style
%matplotlib notebook

In [95]:
def get_data(start_month, start_year, end_month, end_year, symbol, get_dividend = False):
  t = datetime(1970, 1, 1)
  tstart = datetime(start_year, start_month, 1)
  tend = datetime(end_year, end_month+1, 1)
  num_months = (tend.year - tstart.year) * 12 + (tend.month - tstart.month)
  A = []
  B = []
  C = []
  for month in range(num_months):
    tmonth = (tstart + relativedelta(months= +1))
    tickstart = (tstart - t).total_seconds()
    tickend = (tmonth - t).total_seconds()
    url = "https://finance.yahoo.com/quote/"+symbol+"/history?period1="+str(int(tickstart))+"&period2="+str(int(tickend))+"&interval=1d&filter=history&frequency=1d"
    response =  requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    price = soup.find_all('tr', {"C($tertiaryColor) Fz(xs) Ta(end)"})[0]
    iter = 0
    for i in range(len(soup.find_all('td'))):
      
      price = soup.find_all('td')[i].text

      if "*Close price adjusted for splits.**Adjusted close price adjusted for both dividends and splits." in price:
        break
      
      A.append(price)

      if "Dividend" not in price:
        if iter%7 == 6:
          B.append(A)
          A = []
        iter = iter+1

      if "Dividend" in price:
        C.append(A)
        A = []
        iter = iter+6

      tstart = tmonth
  Dividend = pd.DataFrame(C , columns=['Date', 'Dividend'])
  Dividend['Date'] = pd.to_datetime(Dividend['Date'])
  Dividend.sort_values(by=['Date'], inplace=True, ascending=True)
  Dividend.set_index('Date', inplace = True)

  df = pd.DataFrame(B, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])
  df['Date']=pd.to_datetime(df['Date'])
  df.sort_values(by=['Date'], inplace=True, ascending=True)
  df.set_index('Date', inplace = True)

  #for data in [df]:
    #for col in data.columns:
      #data[col] = data[col].apply(lambda x: re.sub(r'[^0-9.]+','',x))

  if get_dividend == True:
    return df, Dividend
  else:
    return df
  

IndentationError: unexpected indent (<ipython-input-95-48ae631fd90e>, line 22)

In [93]:
START_MONTH = 12       #1-12
START_YEAR = 2019      #yyyy

END_MONTH = 6
END_YEAR = 2020

SYMBOL = 'INRJPY=X'

stock_data= get_data(start_month = START_MONTH, start_year = START_YEAR, end_month = END_MONTH, end_year = END_YEAR, symbol = SYMBOL)

In [94]:
stock_data

  

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-12-02,1.5268,1.5312,1.5211,1.5268,1.5268,-
2019-12-03,1.5218,1.5259,1.5122,1.5219,1.5219,-
2019-12-04,1.5146,1.5236,1.5105,1.5145,1.5145,-
2019-12-05,1.5223,1.5293,1.5185,1.5219,1.5219,-
2019-12-06,1.5264,1.5282,1.5202,1.5264,1.5264,-
...,...,...,...,...,...,...
2020-06-15,1.4107,1.4147,1.4056,1.4104,1.4104,-
2020-06-16,1.4145,1.4197,1.4066,1.4145,1.4145,-
2020-06-17,1.4077,1.4112,1.4038,1.4083,1.4083,-
